<a href="https://colab.research.google.com/github/leiyi-ihb/ColabPractics/blob/main/lyProject/03_06_quiz_ComparationModel01_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
'''
前面发现，对于1-3层CNN，只要加了GlobalMaxPool1D就能够训练出某些acc＞0.9的，但是因为同时包含了激活函数为tanh，因此这里把'tanh'改为'relu'试试
整体效果感觉是不错，因此得出结论，这个GlobalMaxPool1D才是关键
'''
import platform
import tensorflow
import keras
from numpy.random import seed
print("Platform:{}".format(platform.platform()))
print("Tensorflow version:{}".format(tensorflow.__version__))
print("Keras version:{}".format(keras.__version__))
import numpy as np
from IPython.display import Image

from keras.models import Model
from keras.layers import Input, Dense, Conv1D, MaxPool1D, GlobalMaxPool1D
from keras.utils import plot_model
from keras.optimizers import Adam, SGD

Platform:Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
Tensorflow version:2.9.2
Keras version:2.9.0


In [3]:
! if [ ! -f "dataset_Li.pkl" ]; then wget https://master.dl.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl ; else echo "Found previous downloaded data."; fi

--2022-11-12 16:19:10--  https://master.dl.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl
Resolving master.dl.sourceforge.net (master.dl.sourceforge.net)... 216.105.38.12
Connecting to master.dl.sourceforge.net (master.dl.sourceforge.net)|216.105.38.12|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl [following]
--2022-11-12 16:19:10--  https://downloads.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Connecting to downloads.sourceforge.net (downloads.sourceforge.net)|204.68.111.105|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cfhcable.dl.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl [following]
--2022-11-12 16:19:11--  https://cfhcable.dl.sourceforge.net/project/summer-re

In [14]:
# Data processing
import pickle
with open("dataset_Li.pkl", "rb") as f:
    data = pickle.load(f)
x_train, y_train = np.array(data['train'][0]), np.array(data['train'][1])
x_test, y_test = np.array(data['test'][0]), np.array(data['test'][1])

print(x_train.shape)

(16000, 1000, 4)


In [ ]:
class CNNModel:  
  def __init__(self):        
    '''optimizer'''
    # self.adam = Adam(lr=0.01)
    # self.sgd = SGD(lr=0.01) 
    # self.model = Sequential()
    pass

  def modelBuild(self, filter, poolSize, kernelSize, unit):
    # model = self.model
    '''model build'''
       
    input = Input(shape=(1000, 4), name='input')
    conv1d_1 = Conv1D(filters=filter, kernel_size=kernelSize, activation='relu', name='conv1d_1')(input)
    pool_1 = MaxPool1D(pool_size=poolSize, name='pool_1')(conv1d_1)
    # conv1d_2 = Conv1D(filters=filter/2, kernel_size=kernelSize, activation='relu', name='conv1d_2')(pool_1)
    # pool_2 = MaxPool1D(pool_size=2, name='pool_2')(conv1d_2)
    goldpool_1 = GlobalMaxPool1D()(pool_1)
    dense_1 = Dense(units=unit, activation='relu', name='dense1')(goldpool_1)
    # dense_2 = Dense(8, activation='relu', name='dense2')(dense_1)
    output = Dense(1, activation='sigmoid', name='output')(dense_1)

    model = Model(inputs=input, outputs=output)
    return model

  def modelCompile(self, opt, m):
    '''model compile'''
    model = m
    model.compile(optimizer=opt,
             metrics = 'acc',
             loss = 'binary_crossentropy')

  def modelFit(self, batSize, m): 
    '''model fix'''
    model = m 
    # model = self.model 
    model.fit(x_train, 
              y_train,
              epochs = 10,
              batch_size = batSize
              )
    
  def modelEvaluate(self, m):
    '''model evaluate'''
    model = m
    # model = self.model
    loss, accuracy1 = model.evaluate(x_train, y_train)
    print("\ntest loss:", loss)
    print('accuracy:', accuracy1)
    
    loss, accuracy2 = model.evaluate(x_test, y_test) #将我们的训练集也评估一下
    print("\ntrain loss:", loss)
    print('accuracy:', accuracy2)
    
    return "test_loss" + "\t" + str(accuracy1) + "\t" + "train_loss" + "\t" + str(accuracy2)


if __name__=='__main__':
  '''
  1, Conv1d: Filters = {32, 16, 3}; Kernel_size = {20, 10, 7, 5, 3}; Activation = {‘relu’, ‘tanh’, ‘linear’}
  2, Pooling1d: Max or Avg pool; Pool size = {8, 4, 2,}
  3, Flatten: Global Max, Global Avg, Flatten
  4, Dense: Units = {32, 64}; Activation = {‘relu’, ‘tanh’, ‘linear’}
  '''
  filter=[2, 4, 8, 16, 32, 64]  
  kernel_size =[20, 10, 7, 5, 3]
  poolSize = [8, 4, 2]
  unit=[4, 8, 16, 32]
  opt=['adam', 'sgd']
  batSize=[4, 8, 10, 12]


  from itertools import product
#   outfile = open("outFileBinaryCrossentropy.txt", "a+")
  loop_val = [filter, kernel_size, poolSize, unit, opt, batSize] #
  for i in product(*loop_val):
    print(i)
    chr_tmp = str(i)
    # if i == (20, 'tanh', 2, 'tanh'):
    outfile = open("outFileBC.txt", "a+")
    obj1 = CNNModel()
    model = obj1.modelBuild(filter=i[0], kernelSize=i[1], poolSize=i[2], unit=i[3])
    obj1.modelCompile(opt=i[4], m = model)
    obj1.modelFit(batSize=i[5], m = model)
    scor = obj1.modelEvaluate(m=model)
    # print(scor)
    outfile.write(chr_tmp + "\t" + scor+"\n")
    outfile.close()

(2, 20, 8, 4, 'adam', 4)
Epoch 1/10
4000/4000 [==============================] - 24s 6ms/step - loss: 0.3107 - acc: 0.8556
Epoch 2/10
4000/4000 [==============================] - 25s 6ms/step - loss: 0.1142 - acc: 0.9609
Epoch 3/10
4000/4000 [==============================] - 26s 7ms/step - loss: 0.1113 - acc: 0.9611
Epoch 4/10
4000/4000 [==============================] - 23s 6ms/step - loss: 0.1107 - acc: 0.9607
Epoch 5/10
4000/4000 [==============================] - 24s 6ms/step - loss: 0.1108 - acc: 0.9606
Epoch 6/10
4000/4000 [==============================] - 23s 6ms/step - loss: 0.1101 - acc: 0.9602
Epoch 7/10
4000/4000 [==============================] - 24s 6ms/step - loss: 0.1100 - acc: 0.9624
Epoch 8/10
4000/4000 [==============================] - 24s 6ms/step - loss: 0.1099 - acc: 0.9613
Epoch 9/10
4000/4000 [==============================] - 23s 6ms/step - loss: 0.1092 - acc: 0.9613
Epoch 10/10
500/500 [==============================] - 5s 10ms/step - loss: 0.1038 - acc: 0.9